# 问题描述

#### 根据所给电影豆瓣评论数据集，通过评论文本和星级的关系，分析文本情感

# 导入外部数据

In [1]:
# 导入数据集
!git clone https://github.com/Computing-Intelligence/datasource

fatal: destination path 'datasource' already exists and is not an empty directory.


In [2]:
# 导入停用词表
!git clone https://github.com/goto456/stopwords.git

Cloning into 'stopwords'...
remote: Enumerating objects: 12, done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 12
Unpacking objects: 100% (12/12), done.


# 数据概览与准备

In [3]:
import numpy as np
import pandas as pd
mov = pd.read_csv('./datasource/movie_comments.csv')

/Users/pollux/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 导入数据后查看数据
mov.head(10)

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1
5,6,https://movie.douban.com/subject/26363254/,战狼2,“犯我中华者，虽远必诛”，吴京比这句话还要意淫一百倍。,1
6,7,https://movie.douban.com/subject/26363254/,战狼2,脑子是个好东西，希望编剧们都能有。,2
7,8,https://movie.douban.com/subject/26363254/,战狼2,三星半，实打实的7分。第一集在爱国主旋律内部做着各种置换与较劲，但第二集才真正显露吴京的野心...,4
8,9,https://movie.douban.com/subject/26363254/,战狼2,开篇长镜头惊险大气引人入胜 结合了水平不俗的快剪下实打实的真刀真枪 让人不禁热血沸腾 特别弹...,4
9,10,https://movie.douban.com/subject/26363254/,战狼2,15/100吴京的冷峰在这部里即像成龙，又像杰森斯坦森，但体制外的同类型电影，主角总是代表个...,1


In [5]:
# 对数据集的整体印象
mov.describe()

,id,link,name,comment,star
count,261497,261497,261497,261495,261497
unique,260150,2761,2760,213970,11
top,16,https://movie.douban.com/subject/1849031/,当幸福来敲门 The Pursuit of Happyness,经典,4
freq,6,396,396,200,43002


In [6]:
# 针对评分 star 列进行查看
mov['star'].value_counts()

4       43002
4       40806
3       33910
5       31947
3       31764
5       27368
2       14299
2       13837
1       12308
1       12255
star        1
Name: star, dtype: int64

In [7]:
# 发现有一行出现特殊值
mov[mov['star'].isin(['star'])] 

,id,link,name,comment,star
568,id,link,name,comment,star


In [8]:
# 删除特殊值
mov=mov.drop(568)

In [9]:
# 查看有无空值
for col in mov.columns:
    print(col, ':', len(mov[mov[col].isnull()]))

id : 0
link : 0
name : 0
comment : 2
star : 0


In [10]:
# 删除空值
mov=mov.dropna()

In [11]:
# 去除重复评论
mov.drop_duplicates(subset=['comment','star'],keep='first',inplace=True)

In [12]:
# 假定我们做二分类，只需要把评论分为：正面和负面 两类
# 设定 1~3星 为负面评论；4~5星 为正面评论
# 增加一个新的列 label 记录正负分类
mov['label'] = mov['star'].map(lambda x: 1 if int(x)>3 else 0)

# 数据预处理

In [13]:
import jieba

# 使用jieba分词来做中文分词
mov['comment'] = mov['comment'].map(str)
mov['cuted'] = mov['comment'].map(lambda x: ' '.join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/95/_015386n3536htxcms976p_40000gn/T/jieba.cache
Loading model cost 0.798 seconds.
Prefix dict has been built succesfully.


In [14]:
mov.head(10)

,id,link,name,comment,star,label,cuted
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1,0,吴京 意淫 到 了 脑残 的 地步 ， 看 了 恶心 想 吐
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2,0,首映礼 看 的 。 太 恐怖 了 这个 电影 ， 不讲道理 的 ， 完全 就是 吴京 在 实...
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2,0,吴京 的 炒作 水平 不输 冯小刚 ， 但小刚 至少 不会 用 主旋律 来 炒作 … 吴京 ...
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4,1,凭良心说 ， 好 看到 不像 《 战狼 1 》 的 续集 ， 完虐 《 湄公河 行动 》 。
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1,0,中二得 很
5,6,https://movie.douban.com/subject/26363254/,战狼2,“犯我中华者，虽远必诛”，吴京比这句话还要意淫一百倍。,1,0,“ 犯 我 中华 者 ， 虽远必 诛 ” ， 吴京 比 这句 话 还要 意淫 一百倍 。
6,7,https://movie.douban.com/subject/26363254/,战狼2,脑子是个好东西，希望编剧们都能有。,2,0,脑子 是 个 好 东西 ， 希望 编剧 们 都 能 有 。
7,8,https://movie.douban.com/subject/26363254/,战狼2,三星半，实打实的7分。第一集在爱国主旋律内部做着各种置换与较劲，但第二集才真正显露吴京的野心...,4,1,三星 半 ， 实打实 的 7 分 。 第一集 在 爱国 主旋律 内部 做 着 各种 置换 与...
8,9,https://movie.douban.com/subject/26363254/,战狼2,开篇长镜头惊险大气引人入胜 结合了水平不俗的快剪下实打实的真刀真枪 让人不禁热血沸腾 特别弹...,4,1,开篇 长镜头 惊险 大气 引人入胜 结合 了 水平 不俗 的 快 剪下 实打实 的 真刀...
9,10,https://movie.douban.com/subject/26363254/,战狼2,15/100吴京的冷峰在这部里即像成龙，又像杰森斯坦森，但体制外的同类型电影，主角总是代表个...,1,0,15 / 100 吴京 的 冷峰 在 这部 里 即 像 成龙 ， 又 像杰 森斯坦 森 ， ...


In [15]:
# 设定输入和输出
# 大写X表示输入：评论
# 小写y表示输出：正面/负面
X = mov['cuted']
y = mov['label']

# 划分训练集和测试集

In [16]:
# 采用 sklearn 库的 train_test_split 划分数据集和测试集。
# 默认25%做为测试集
from sklearn.model_selection import train_test_split

# 划分训练集和测试集
# random_state 指定随机数种子
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# 查看训练集和测试集
print('Train datasets : ', X_train.shape)
print('Test  datasets : ', X_test.shape)

Train datasets :  (163220,)
Test  datasets :  (54407,)


# 文本实例化

In [17]:
# sklearn 库的 CountVectorizer 实现文本的词袋表示，把中文文本实例化
from sklearn.feature_extraction.text import CountVectorizer

# 变换器
vect = CountVectorizer()
vect.fit(X_train)

# vocabulary_ 查看词表，即每个单词对应的索引
# 查看词表数量
print(len(vect.vocabulary_))
# 打印词表
print(vect.vocabulary_)

126166
{'黑暗': 125658, '舞者': 102973, '极品': 74822, 'death': 4075, 'dancer': 3992, '第五部': 95882, '终于': 98097, '终结': 98146, '以前': 22670, '人类': 21812, '变形金刚': 35781, '力挫': 32089, '霸天虎': 121691, '老套路': 100011, '世界观': 17153, '引入': 55715, '电影': 89175, '中来': 18289, '昆塔沙': 70341, '一族': 13542, '浮出': 81605, '水面': 79140, '期待': 73116, '全新': 27269, '世界': 17136, '神片': 94061, '怎么': 58727, '这么': 113538, '低分': 23977, '时间': 70280, '灰烬': 84105, '最后': 71975, '一个': 12869, '故事': 67676, '简直': 96099, '两星': 17550, '片中': 85861, '风景': 123101, '观影': 107587, '顺序': 122632, '决定': 29157, '评价': 108729, '这年头': 113679, '面瘫': 122139, '棒子': 76300, '不能': 16637, '拍电影': 64513, '演技': 83458, '白甜': 90617, '女当': 46615, '花瓶': 103543, '场面': 41754, '华丽': 33643, '有种': 72793, '史诗': 36868, '感觉': 60701, '但是': 23860, '由于': 89054, '故事情节': 67680, '单一': 33755, '打斗': 62849, '过瘾': 113143, '格里菲斯': 75825, '真的': 92569, '夺目': 45951, '想起': 60274, '第三部': 95817, '惨状': 60083, '顿时': 122722, '觉得': 107689, '干什么': 54252, '无罪': 69707, '打工': 62786, '女说': 46

In [18]:
# import pandas as pd

# 采用 transform 方法创建词袋的稀疏矩阵
# 矩阵中每个特征对应词表中的单词
# 如果没有这个单词会用0进行填充。
words_matrix = pd.DataFrame(vect.transform(X).toarray(),
                            columns=vect.get_feature_names())
words_matrix.head()

,00,000,0000020529,0001,001,0014221b798a,0015c55db73d,005,006,007,...,龚雪,龚雪则,龚雪好,龚雪演,龚雪真,龟头,龟毛,龟派,龟甲,龟苓膏
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 构建模型

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# 通过构建 LogisticRegression 逻辑回归分类器来拟合训练集数据
# 使用 cross_val_score 交叉验证对 LogisticRegression 进行评估模型的性能

# 交叉验证评估模型
scores = cross_val_score(LogisticRegression(solver='liblinear'),
                         vect.transform(X_train), y_train, cv=5)
print('平均交叉验证准确率：{:.1f}%'.format(np.mean(scores)*100))

/Users/pollux/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


平均交叉验证准确率：71.4%


In [ ]:
# 得到 平均交叉验证准确率为 71.4%
# 对于二分类模型比较合理的
# 在多维特征的数据进行拟合分类，通常逻辑回归会有不错的效果

In [25]:
# 对评论去除停用词
# 这里使用哈工大的停用词表
# 经实验，哈工大停用词表.txt 中，未包含少量停用词
# 因此加入之后，另存为 哈工大停用词表2.txt
def stopwords_list():
    with open('./stopwords/哈工大停用词表2.txt') as f:
        lines = f.readlines()
        result = [i.strip('\n') for i in lines]
    return result

stopwords = stopwords_list()

In [28]:
# 重新构建单词矩阵
# max_df：表示舍弃最频繁的单词
# min_df：表示每个词必须要在3个评论中出现
# stop_words：对于中文需要指定停用词列表
# 使用正则表达式去掉所有数字

vect = CountVectorizer(max_df=0.85, min_df=3, stop_words=stopwords,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b')

vect.fit(X_train)

words_matrix = pd.DataFrame(vect.transform(X_train).toarray(),
                            columns=vect.get_feature_names())
words_matrix.head(10)

,__,___,____,a4,a5,a8,aa,aac,aaliyah,aaron,...,龙虎,龙虎榜,龙虎风云,龙虾,龙门,龙门客栈,龚格尔,龚雪,龟头,龟派
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# 去除停用词后，继续训练模型
lr = LogisticRegression(solver='liblinear')

lr.fit(vect.transform(X_train), y_train)

print('测试集准确率：{:.1f}%'.format(lr.score(vect.transform(X_test), y_test)*100))

/Users/pollux/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


测试集准确率：71.2%


In [ ]:
# 可见，去除停用词对于此数据集的词袋模型正确率并无显著提升

In [36]:
# 用 TF-IDF 评估字词对于评论文本的重要程度
# 调用 sklearn 库中两个类中实现 TF-IDF 方法
# TfidfTransformer 接受 CountVectorizer 生成的稀疏矩阵并将其变换
# TfidfVectorizer 接受文本数据并完成词袋特征提取与 TF-IDF 变换。

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(min_df=3), LogisticRegression(solver='liblinear'))
pipe.fit(X_train, y_train)
scores = cross_val_score(pipe, X_train, y_train, cv=5)
print('平均交叉验证准确率：{:.1f}%'.format(np.mean(scores)*100))

平均交叉验证准确率：72.6%


In [39]:
# 用 TF-IDF 方法相对于仅统计词数的正确率有所提高
# 同时可以查看 TF-IDF 找到的最重要的单词
# TF-IDF 较低的词要么经常出现，要么就是很少出现
# TF-IDF 较大的词往往在评论中经常出现
vectorizer = pipe.named_steps['tfidfvectorizer']
# 找到每个特征中最大值
max_value = vectorizer.transform(X_train).max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# 获取特征名称
feature_names = np.array(vectorizer.get_feature_names())

print("TF-IDF 较低的词：\n{}".format(feature_names[sorted_by_tfidf[:10]]))
print()
print("TF-IDF 较高的词：\n{}".format( feature_names[sorted_by_tfidf[-10:]]))

TF-IDF 较低的词：
['智武' '以抗' '国富民强' '外敌' 'zo' 'saldana' 'quinto' 'zachary' '我民' '之见']

TF-IDF 较高的词：
['看热闹' '次奥' '细细' 'sad' '刘心悠' '浮云' '强力' '圣母' '捆绑' '演员表']


# 模型评估

In [48]:
from sklearn import metrics

# 预测值
y_pred = pipe.predict(X_test)

print('测试集准确率：{:.1f}%'.format(metrics.accuracy_score(y_test, y_pred)*100))
print('测试集准确率：{:.1f}%'.format(pipe.score(X_test, y_test)*100))

pred = metrics.confusion_matrix(y_test, y_pred)
pred

测试集准确率：73.4%
测试集准确率：73.4%


array([[15975,  8951],
       [ 5526, 23955]])

In [55]:
# 模型混淆矩阵上看到

print('模型在负评价上的准确率为：{:.1f}%'.format(pred[0,0]/(pred[0,0]+pred[1,0])*100))
      
print('模型在正评价上的准确率为：{:.1f}%'.format(pred[1,1]/(pred[0,1]+pred[1,1])*100))

模型在负评价上的准确率为：74.3%
模型在正评价上的准确率为：72.8%
